In [39]:
import numpy as np
from scipy.spatial import KDTree
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from numba import njit, prange

In [2]:
tmp = "./read_bin_bhdisk 1.9840100000000001e+02 6.4000000000000043e+01 3.9680000000000001e+02 6.4000000000000004e+00 6.4000000000000004e+00 6.4000000000000004e+00 41 37 39 -979427380"

In [3]:
parts = tmp.split()[1:]

In [4]:
np.float64(parts)

array([ 1.9840100e+02,  6.4000000e+01,  3.9680000e+02,  6.4000000e+00,
        6.4000000e+00,  6.4000000e+00,  4.1000000e+01,  3.7000000e+01,
        3.9000000e+01, -9.7942738e+08])

In [5]:
xarr = np.arange(1, 10, 1, dtype=np.float64)
yarr = np.arange(-5, 5, 1, dtype=np.float64)
zarr = np.arange(3, 7, 1, dtype=np.float64)
cartgrid = np.array(np.meshgrid(xarr, yarr, zarr)).T.reshape(-1, 3)

In [6]:
cartgrid[23]

array([ 3., -2.,  3.])

In [7]:
tree = KDTree(cartgrid)

In [8]:
foo = np.array([[2.1, 3.8, 3.1], [3.1, -1.9, 2.9]])
d, i = tree.query(foo)

In [9]:
i

array([19, 23])

In [10]:
cartgrid[i]

array([[ 2.,  4.,  3.],
       [ 3., -2.,  3.]])

In [11]:
ps = np.zeros(5)

In [12]:
ps

array([0., 0., 0., 0., 0.])

In [13]:
df = pd.DataFrame(cartgrid, columns=['x','y', 'z'])

In [14]:
df

,x,y,z
0,1.0,-5.0,3.0
1,1.0,-4.0,3.0
2,1.0,-3.0,3.0
3,1.0,-2.0,3.0
4,1.0,-1.0,3.0
...,...,...,...
355,9.0,0.0,6.0
356,9.0,1.0,6.0
357,9.0,2.0,6.0
358,9.0,3.0,6.0


In [15]:
df['f'] = df.x*df.y/df.z

In [16]:
df

,x,y,z,f
0,1.0,-5.0,3.0,-1.666667
1,1.0,-4.0,3.0,-1.333333
2,1.0,-3.0,3.0,-1.000000
3,1.0,-2.0,3.0,-0.666667
4,1.0,-1.0,3.0,-0.333333
...,...,...,...,...
355,9.0,0.0,6.0,0.000000
356,9.0,1.0,6.0,1.500000
357,9.0,2.0,6.0,3.000000
358,9.0,3.0,6.0,4.500000


In [17]:
df['g'] = df.x**2 + df.y**2 + df.z**2

In [18]:
df['h'] = 3*df.x*df.y*df.z

In [19]:
data = df.to_numpy()

In [20]:
from scipy.interpolate import RBFInterpolator, BarycentricInterpolator

In [21]:
interp = np.zeros(27)
interp[0:3] = np.array([1.3, 3.3, 4.9])

In [22]:
interp

array([1.3, 3.3, 4.9, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       0. ])

In [23]:
_, i = tree.query(interp[0:3], k=9, workers=-1)

In [24]:
data[i][:, 0]

array([1., 2., 1., 2., 1., 1., 2., 1., 2.])

In [25]:
np.array([
    BarycentricInterpolator(data[i][:, 0], data[i][:, j], interp[0]) for j in range(4, 27)
], dtype=np.float64)

TypeError: tuple indices must be integers or slices, not numpy.float64

In [ ]:
def cartesiantospherical(arr):
    r = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2 + arr[:, 2]**2)
    theta = np.where(r != 0.0, np.arccos(np.clip(arr[:, 2] / r, -1.0, 1.0)), 0.0)
    phi = np.arctan2(arr[:, 1], arr[:, 0])
    new_grid = np.column_stack((r, theta, phi))
    new_grid = np.where(abs(new_grid) < 1e-14, np.float64(0), new_grid)
    return new_grid

In [ ]:
vals = [barycentric_interpolate(data[:, 0], data[:, i], rp) for i in range(3, 27)]

In [26]:
k = np.arange(0, 27, 1)

In [27]:
k

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26])

In [28]:
k[26]

np.int64(26)

In [29]:
intp = RBFInterpolator(data[i, 0:3], data[i, 3:], kernel="linear")

In [30]:
intp(np.array([[1.3, 3.3, 4.9]]))[0]

array([ 0.87105629, 36.87954131, 62.39317888])

In [31]:
data[23, :3]

array([ 3., -2.,  3.])

In [44]:
def cartesiantospherical(arr):
    r = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2 + arr[:, 2]**2)
    theta = np.where(r != 0.0, np.arccos(np.clip(arr[:, 2] / r, -1.0, 1.0)), 0.0)
    phi = np.arctan2(arr[:, 1], arr[:, 0])
    new_grid = np.column_stack((r, theta, phi))
    new_grid = np.where(abs(new_grid) < 1e-14, np.float64(0), new_grid)
    return new_grid

@njit(parallel=True)
def cartesiantospherical_parallel(arr):
    n = arr.shape[0]
    new_grid = np.zeros((n, 3), dtype=np.float64)
    for i in prange(n):  # Parallel loop
        x, y, z = arr[i]
        r = np.sqrt(x**2 + y**2 + z**2)
        theta = np.arccos(z / r) if r != 0 else 0.0
        phi = np.arctan2(y, x)
        new_grid[i, 0] = 0 if np.abs(r) < 1e-14 else r
        new_grid[i, 1] = 0 if np.abs(theta) < 1e-14 else theta
        new_grid[i, 2] = 0 if np.abs(phi) < 1e-14 else phi
    return new_grid

def process_line(line):
    xmin, ymin, zmin, dx, dy, dz, nx, ny, nz, MPI_ID = map(np.float64, line.split()[1:]) 
    xarr = np.arange(xmin, xmin + nx*dx, dx)
    yarr = np.arange(ymin, ymin + ny*dy, dy)
    zarr = np.arange(zmin, zmin + nz*dz, dz)
    cartgrid = np.array(np.meshgrid(xarr, yarr, zarr)).T.reshape(-1, 3)
    sphericalgrid = cartesiantospherical_parallel(cartgrid)
    output_file = f"CTS_bin-proc{int(MPI_ID)}.d"
    return [cartgrid, sphericalgrid, nx, ny, nz, output_file]



In [45]:
lines = []
with open("/home/shreyjam/scratch/research/thesis/BlackHoleDiskProject/src/initialdata/grids_bh_disk_patrik", "r") as f:
    lines = [line for line in f]

In [46]:
lines

['./read_bin_bhdisk 1.9840100000000001e+02 6.4000000000000043e+01 3.9680000000000001e+02 6.4000000000000004e+00 6.4000000000000004e+00 6.4000000000000004e+00 41 37 39 -979427380  \n',
 './read_bin_bhdisk 1.1520100000000002e+02 4.1600000000000037e+01 2.2400000000000000e+02 3.2000000000000002e+00 3.2000000000000002e+00 3.2000000000000002e+00 45 40 47 1510771035  \n',
 './read_bin_bhdisk 5.7601000000000013e+01 2.0800000000000036e+01 1.1200000000000001e+02 1.6000000000000001e+00 1.6000000000000001e+00 1.6000000000000001e+00 45 40 47 -877886181  \n',
 './read_bin_bhdisk 2.8801000000000013e+01 1.0400000000000036e+01 5.6000000000000007e+01 8.0000000000000004e-01 8.0000000000000004e-01 8.0000000000000004e-01 45 40 47 1232951467  \n',
 './read_bin_bhdisk 1.4401000000000012e+01 5.2000000000000357e+00 2.8000000000000000e+01 4.0000000000000002e-01 4.0000000000000002e-01 4.0000000000000002e-01 45 40 47 380747062  \n',
 './read_bin_bhdisk 7.2010000000000121e+00 2.6000000000000356e+00 1.4000000000000

In [47]:
with ThreadPoolExecutor() as executor:
    grid_data = list(executor.map(process_line, lines))


In [48]:
grid_data

[[array([[198.401,  64.   , 396.8  ],
         [198.401,  70.4  , 396.8  ],
         [198.401,  76.8  , 396.8  ],
         ...,
         [454.401, 281.6  , 646.4  ],
         [454.401, 288.   , 646.4  ],
         [454.401, 294.4  , 646.4  ]]),
  array([[4.48228956e+02, 4.83739586e-01, 3.12040649e-01],
         [4.49187441e+02, 4.87785547e-01, 3.40977365e-01],
         [4.50234868e+02, 4.92152503e-01, 3.69331961e-01],
         ...,
         [8.38815706e+02, 6.90998148e-01, 5.54791242e-01],
         [8.40985867e+02, 6.94112540e-01, 5.64903407e-01],
         [8.43199021e+02, 6.97260257e-01, 5.74887597e-01]]),
  np.float64(41.0),
  np.float64(37.0),
  np.float64(39.0),
  'CTS_bin-proc-979427380.d'],
 [array([[115.201,  41.6  , 224.   ],
         [115.201,  44.8  , 224.   ],
         [115.201,  48.   , 224.   ],
         ...,
         [256.001, 160.   , 371.2  ],
         [256.001, 163.2  , 371.2  ],
         [256.001, 166.4  , 371.2  ]]),
  array([[2.55299492e+02, 5.00378815e-01, 3.4653609